当我们需要训练一个深层的神经网络，比如10层，每一层都含有数百个神经元，通过数十万个链接相连，这时，就会遇到如下问题；

- (1)遇到很诡异的梯度消失或梯度爆炸问题，它们会影响深度神经网络，从而导致底层训练困难
- (2)对于这么庞大的一个网络，训练速度会非常慢
- (3)一个数百万参数的模型会很容易出现过拟合风险

面对这些问题，你会怎么办？

<b>简单描述下梯度消失和梯度爆炸的问题：</b>

反向传播算法是从输出层反向作用到输入层，在整个过程中传播误差梯度，并根据该误差梯度来修正各个神经网络层对应的权重<之前的博客已经实现了反向传播算法>

不幸的是，梯度经常会随着算法进展到更低层时变得越来越小，导致底层的权重基本没有更新变化，从而导致训练收敛效果不好，这就是梯度消失问题

梯度爆炸就是梯度消失相反现象，梯度会越来越大，导致很对层的权重值疯狂增大，使得算法发散

为了解决梯度消失和梯度爆炸的问题，经过相当长的一段时间研究，直到2010年左右Xavier Glorot和Yoshua Bengio提出了使用逻辑S激活函数和权重初始化技术的观点，使得对于这个问题的研究有了重大突破

# 权重初始化

在Xavier Glorot和Yoshua Bengio的论文中提出：要让信号在两个方向都正确流动，不消亡、爆炸和稀释，就需要保持每一层的输入和输出的方差一致，并且需要在反向流动过某一层时，前后的方差也要一致。事实上，是很难保证的，除非一层有相同数量的输入和输出链接，当然他们也提出了一种折中方案:链接权重必须按照以下公式进行随机初始化，即Xavier初始化。注：正太分布中均值为0，$n_{inputs}和n_{outputs}$是权重被初始化层输入和输出的链接数

## 逻辑函数Xavier初始化

（1）均匀分布
$$r = \sqrt{\frac{6}{n_{inputs} + n_{outputs}}}$$

(2) 正太分布
$$\sigma = \sqrt{\frac{2}{n_{inputs} + n_{outputs}}}$$

## 双曲正切函数Xavier初始化

（1）均匀分布
$$r = \sqrt[4]{\frac{6}{n_{inputs} + n_{outputs}}}$$

(2) 正太分布
$$\sigma = \sqrt[4]{\frac{2}{n_{inputs} + n_{outputs}}}$$

## ReLU(及变种)Xavier初始化

（1）均匀分布
$$r = \sqrt[\sqrt{2}]{\frac{6}{n_{inputs} + n_{outputs}}}$$

(2) 正太分布
$$\sigma = \sqrt[\sqrt{2}]{\frac{2}{n_{inputs} + n_{outputs}}}$$

## He初始化

He初始化的思想是:假定每一层有一半的神经元被激活，另一半为零，所以要保持方差不变，只需要在Xavier的基础上再除以2
$$tf.Variable(np.random.randn(n_{inputs},n_{outputs})) / np.sqrt(n_{inputs}/2)$$

# 选择适当的激活函数

<img src='./datasets/active_fn.png'>

上图绘制神经网络中常用的激活函数曲线和对应的概率分布曲线，更多可以看我之前的博客<神经网络之激活函数和损失函数>

上图中没有LeakyReLU函数，先补充下$$LeakyRelu_a(z) = max(az,z)$$

Glorot和Bengio在2010年的论文中提到一个观点，梯度消失/梯度爆炸一部分原因是选错了激活函数

<b>究竟该选择哪一种激活函数呢？</b>通常来说ELU函数 > leaky ReLU(和它的变种) > ReLU > tanh > 逻辑函数。如果关心运行时的性能，哪可以选择LeakyReLU函数，而不是ELU函数。如果不想改变别的超参数，就只能建议$\alpha$的默认值(LeakyreLU函数是0.01，ELU是1)。如果有多余的时间和计算能力，可以使用交叉验证去评估别的激活函数，特别是如果网络过度拟合，可以使用RRelu函数，又或者是针对大的训练集使用PReLU

# 批量归一化

尽管使用Xavier/He权重初始化和ELU(RELU的任一变种)可以很明显地在训练初期降低梯度消失/梯度爆炸问题,但还是不能保证在训练过程中不会再出现这些问题，Sergey Ioffe和Christian Szegedy提出了批量归一化(BN)的技术来解决梯度消失/爆炸问题

该技术包括在每一层激活函数之前在模型里加一个操作，简单零中心化和归一化输入，之后在再通过每层的两个新参数(一个为了缩放，一个为了移动)缩放和移动的结果。换句话说，这个操作让模型学会了最佳规模和每层输入的平均值

## 批量归一化原理

(1)$\mu_B = \frac{1}{m_B}\sum_{i=1}^{m_B}x^{(i)}$ #经验平均值，评估整个小批量B

(2)$\theta_B = \frac{1}{m_B}\sum_{i=1}^{m_b}(x^{(i)} - \mu_B)^2$ #评估整个小批量B的方差

(3)$x_{(i)}^* = \frac{x^{(i)} - \mu_B}{\sqrt{\theta_B^2+\xi}}$#零中心化和归一化

(4)$z^{(i)} = \lambda x_{(i)}^* + \beta$#将输入进行缩放和移动

在测试期间，没有小批量的数据来计算经验平均值和标准方差，所有可以简单地用整个训练集的平均值和标准方差来代替，在训练过程中可以用变动平均值有效计算出来

<b>但是</b>，批量归一化的确也给模型增加了一些复杂度和运行代价，使得神经网络的预测速度变慢，所以如果需要快速预测，可能需要在进行批量归一化之前先检查一下ELU+He初始化的表现如何

# 梯度裁剪

# 重用模型

# 冻结底层

# 缓存冻结层

# 选择适当的优化器